# Experimental Design
***
## Contrast Choice

Import the necessary packages and code for this application of the framework.

In [1]:
%matplotlib inline

import numpy as np

from refnx.reflect import SLD, Slab
from refnx.analysis import Parameter

from hogben.optimise import Optimiser
from hogben.visualise import contrast_choice_single, contrast_choice_double

# Reduce size of plots for the notebook.
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['figure.dpi'] = 100

Next, define the structure that we want to find the optimal contrasts for. The code below defines a model in [refnx](https://refnx.readthedocs.io/) describing a 1,2-dimyristoyl-sn-glycero-3-phosphocholine (DMPC) bilayer. It inherits from the BaseLipid class which contains the methods for calculating the Fisher information.

In [2]:
from hogben.models.base import BaseLipid

class BilayerDMPC(BaseLipid):
    def __init__(self):
        self.name = 'DMPC_bilayer'

        self.si_sld      = 2.073
        self.sio2_sld    = 3.41
        self.dmpc_hg_vol = 320.9
        self.dmpc_tg_vol = 783.3
        self.dmpc_hg_sl  = 6.41e-4
        self.dmpc_tg_sl  = -3.08e-4
        self.water_vol   = 30.4

        # Calculate the SLD of the tailgroup.
        self.tg_sld = (self.dmpc_tg_sl / self.dmpc_tg_vol) * 1e6

        # Define the parameters of the model.
        self.si_rough      = Parameter(2,     'Si/SiO2 Roughness',      (1,8))
        self.sio2_thick    = Parameter(14.7,  'SiO2 Thickness',         (5,20))
        self.sio2_rough    = Parameter(2,     'SiO2/DMPC Roughness',    (1,8))
        self.sio2_solv     = Parameter(0.245, 'SiO2 Hydration',         (0,1))
        self.dmpc_apm      = Parameter(49.9,  'DMPC Area Per Molecule', (30,60))
        self.bilayer_rough = Parameter(6.57,  'Bilayer Roughness',      (1,8))
        self.bilayer_solv  = Parameter(0.074, 'Bilayer Hydration',      (0,1))
        self.hg_waters     = Parameter(3.59,  'Headgroup Bound Waters', (0,20))

        self.params = [self.si_rough,
                       self.sio2_thick,
                       self.sio2_rough,
                       self.sio2_solv,
                       self.dmpc_apm,
                       self.bilayer_rough,
                       self.bilayer_solv,
                       self.hg_waters]

        # Vary all of the parameters defined above.
        for param in self.params:
            param.vary=True

    def _using_conditions(self, contrast_sld, underlayers=None):
        """Creates a structure representing the bilayer measured using a
           contrast of given `contrast_sld`.

        Args:
            contrast_sld (float): SLD of contrast to simulate.
            underlayer (list): thicknesses and SLDs of underlayers to add.

        Returns:
            refnx.reflect.Structure: structure in given contrast.

        """
        # We do not need to consider underlayers for this example.
        assert underlayers is None
        
        # Calculate the SLD of the headgroup with the given contrast SLD.
        hg_sld = contrast_sld*0.27 + 1.98*0.73

        vol_hg = self.dmpc_hg_vol + self.hg_waters*self.water_vol
        hg_thick = vol_hg / self.dmpc_apm
        tg_thick = self.dmpc_tg_vol / self.dmpc_apm

        substrate = SLD(self.si_sld)
        solution  = SLD(contrast_sld)(rough=self.bilayer_rough)

        sio2     = Slab(self.sio2_thick, self.sio2_sld, self.si_rough,      vfsolv=self.sio2_solv)
        inner_hg = Slab(hg_thick,        hg_sld,        self.sio2_rough,    vfsolv=self.bilayer_solv)
        outer_hg = Slab(hg_thick,        hg_sld,        self.bilayer_rough, vfsolv=self.bilayer_solv)
        tg       = Slab(tg_thick,        self.tg_sld,   self.bilayer_rough, vfsolv=self.bilayer_solv)
        solution  = SLD(contrast_sld)(rough=self.bilayer_rough)

        return substrate | sio2 | inner_hg | tg | tg | outer_hg | solution
    
    # For this notebook we do not need to load any measured data.
    def _create_objectives(self):
        pass

Define the inputs to the contrast selection code.

In [3]:
# Path to directory to save results to.
save_path = './results'

# Specify the model.
sample = BilayerDMPC()

# Number of points and times to simulate for each angle.
angle_times = [(0.7, 100, 10), (2.3, 100, 40)] # (Angle, Points, Time)

# Contrast SLDs to consider.
contrast_range = np.linspace(-0.56, 6.36, 200)

***
### Single Contrast
We can investigate the choice of contrast SLD for single contrast choices. First assuming only D2O has been measured, then only H2O, and finally D2O and H2O.

In [4]:
# Single contrast choice, assuming D2O was first measured.
contrast_1 = contrast_choice_single(sample, contrast_range, [6.36], angle_times, save_path, 'D2O')
print('Contrast choice with D2O: {}'.format(round(contrast_1, 2)))

>>> 0/200
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]
[(0.7, 100, 10), (2.3, 100, 40)]



KeyboardInterrupt



In [ ]:
# Single contrast choice, assuming H2O was first measured.
contrast_2 = contrast_choice_single(sample, contrast_range, [-0.56], angle_times, save_path, 'H2O')
print('Contrast choice with H2O: {}'.format(round(contrast_2, 2)))

In [ ]:
# Single contrast choice, assuming D2O and H2O were measured.
contrast_3 = contrast_choice_single(sample, contrast_range, [-0.56, 6.36], angle_times, save_path, 'H2O_D2O')
print('Contrast choice with D2O & H2O: {}'.format(round(contrast_3, 2)))

***
### Multiple Contrasts
We can also consider the choice of two contrasts simultaneously. In this case, we assume no prior measurement. Check the results folder to see different views of this 3D plot.

In [ ]:
contrast_range = np.linspace(-0.55, 6.36, 20) # Define a smaller range than before.
contrast_pair = contrast_choice_double(sample, contrast_range, angle_times, save_path)
print('Contrast pair: {}'.format(contrast_pair))

***
### Optimisation

As we go beyond selecting two contrasts at once, visualising the optimisation space becomes difficult. Additionally, we have not considered how our total counting time should be split between each contrast. Instead of the brute force approach used above, we can apply an established optimisation algorithm to find the optimal solution for us. Here we apply the differential evolution algorithm to find the optimal choice of contrasts and counting time splits between them.

Due to [issues](https://docs.python.org/2.7/library/multiprocessing.html#using-a-pool-of-workers) with the multiprocessing module and interactive interpreters on Windows, the optimisation function must be in a separate file. Here we import the same bilayer sample, as defined above, from a different file to fix this issue. If running on other operating systems, this cell can be ignored. If on Windows and you wish to use the model defined in this notebook, you can set `workers=1` but this will be very slow!

In [5]:
from hogben.models.bilayers import BilayerDMPC
sample = BilayerDMPC()

In [6]:
optimiser = Optimiser(sample)
num_contrasts = 3
angle_splits = [(0.7, 100, 0.2), (2.3, 100, 0.8)]

contrasts, splits, _ = optimiser.optimise_contrasts(num_contrasts, angle_times, workers=-1, verbose=False)

times = np.array(splits) * 100 # Total counting time of 100

print('Contrasts: {}'.format(np.round(contrasts, 2)))
print('Times: {}'.format(np.round(times, 1)))

[(0.7, 100, 966.8645721957064), (2.3, 100, 3867.4582887828255)]
[(0.7, 100, 966.8645721957064), (2.3, 100, 3867.4582887828255)]
[(0.7, 100, 966.8645721957064), (2.3, 100, 3867.4582887828255)]
[(0.7, 100, 1043.181181348305), (2.3, 100, 4172.72472539322)]
[(0.7, 100, 1043.181181348305), (2.3, 100, 4172.72472539322)]
[(0.7, 100, 1043.181181348305), (2.3, 100, 4172.72472539322)]
[(0.7, 100, 431.2812861441279), (2.3, 100, 1725.1251445765115)]
[(0.7, 100, 431.2812861441279), (2.3, 100, 1725.1251445765115)]
[(0.7, 100, 431.2812861441279), (2.3, 100, 1725.1251445765115)]
[(0.7, 100, 1126.2864165866922), (2.3, 100, 4505.145666346769)]
[(0.7, 100, 1126.2864165866922), (2.3, 100, 4505.145666346769)]
[(0.7, 100, 1126.2864165866922), (2.3, 100, 4505.145666346769)]
[(0.7, 100, 1071.4201606554097), (2.3, 100, 4285.680642621639)]
[(0.7, 100, 1071.4201606554097), (2.3, 100, 4285.680642621639)]
[(0.7, 100, 1071.4201606554097), (2.3, 100, 4285.680642621639)]
[(0.7, 100, 1362.9975664181381), (2.3, 100, 54

Process ForkPoolWorker-9:
Process ForkPoolWorker-7:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Traceback (most recent call last):

KeyboardInterrupt



As can be seen, the results strongly agree with our findings when considering contrasts individually.